In [69]:
import pandas as pd
import json

In [70]:
df = pd.read_excel('05_Sentimiento.xlsx')

In [71]:
umbral_confianza = 0.5
df['orac_totales'] = None
df['orac_validas'] = None
df['estrellas_totales'] = None
df['estrellas_neto'] = None

for index, row in df.iterrows():
    evaluaciones = pd.DataFrame(json.loads(row.eval))
    validas = evaluaciones[evaluaciones['puntaje']>=umbral_confianza]
    #print(f"Indice {index} tiene {orac_validas} oraciones validas que suman {estrellas_totales} estrellas para un resultado neto de {estrellas_neto}")
    df.at[index,'orac_totales'] = len(evaluaciones)
    df.at[index,'orac_validas'] = len(validas)
    if len(validas) > 0:
        df.at[index,'estrellas_totales'] = sum(validas['estrellas'])
        df.at[index,'estrellas_neto'] = sum(validas['estrellas'])-len(validas)*3    

In [72]:
intervalos = [-10, -3, -1, 1, 3]
labels = ['Inconcluso', 'Negativo', 'Neutro', 'Positivo']
df['eval_final'] = None
df['label_final'] = None

for index, row in df.iterrows():
    if row.orac_validas > 0:
        df.at[index,'eval_final'] = (row.estrellas_neto) / row.orac_validas
    else:
        df.at[index,'eval_final'] = -10

df['label_final'] = pd.cut(df['eval_final'], bins = intervalos, labels = labels, include_lowest = True, right = True)
df['label_final'] = df['label_final'].astype('category')

In [73]:
df.drop('Unnamed: 0.2', axis = 1, inplace=True)
df.drop('Unnamed: 0.1', axis = 1, inplace=True)
df.drop('Unnamed: 0', axis = 1, inplace=True)

In [74]:
df.to_excel('06_Analisis.xlsx')